#  Проект: Прогнозирование оттока клиентов Банка

## Задача

Из «Бета-Банка» стали уходить клиенты.
Моя задача - сохранить текущих клиентов, потому что это дешевле, чем привлекать новых. Я должен спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Мне предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

## План
1. Загрузим и подготовим данные.
2. Исследуем баланс классов, обучим модель без учёта дисбаланса.
3. Улучшим качество модели, учитывая дисбаланс классов. Обучим разные модели и найдём лучшую.
4. Проведем финальное тестирование.



## Описание данных

Признаки

* RowNumber — индекс строки в данных

* CustomerId — уникальный идентификатор клиента

* Surname — фамилия

* CreditScore — кредитный рейтинг

* Geography — страна проживания

* Gender — пол

* Age — возраст

* Tenure — сколько лет человек является клиентом банка

* Balance — баланс на счёте

* NumOfProducts — количество продуктов банка, используемых клиентом

* HasCrCard — наличие кредитной карты

* IsActiveMember — активность клиента

* EstimatedSalary — предполагаемая зарплата

Целевой признак

* Exited — факт ухода клиента

# 1. Подготовка данных

### 1.1 Смотрим данные

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
rnd_state = 12345 #Значение Random_state, которое мы будет использваоть 

In [4]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv("C:/Users/gorok/Desktop/Churn.csv")

In [5]:
print(data.shape)
data.head(25)

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В нашем датасете 10.000 строк и 14 столбцов, в столбце Tenure есть пропуски, далее выясним, что они могут означать

В следующем шаге изменим названия, поменяем типы данных, попробуем заполнить пропуски

### 1.2 Предобработка

Изменим названия стобцов

In [7]:
data.columns = data.columns.str.lower()
data.rename(columns={
    'rownumber':'row_number', 
    'customerid':'customer_id', 
    'creditscore':'credit_score', 
    'numofproducts':'num_of_products', 
    'hascrcard':'has_cr_card', 
    'isactivemember':'is_active_member', 
    'estimatedsalary':'estimated_salary'
    
}, inplace=True)
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Посмотрим на пропуски

In [8]:
data['tenure'].value_counts(dropna=False)

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

Вполне возможно, что пропуски - это какой-то особый статус пользователей, которые либо совсем недавно пришли в банк, либо наоборот долго пользуются им, возможно для них есть особое обозначение(нечисловое) 

Замена типов данных, нигде не пригодилась 

### 1.3 Отберем признаки

После освновной предобработки, приступим к подготовке признаков для последующего обучения

In [9]:
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


В наши признаки нет никакого смысла добавлять номер строки, айди покупателя, фамилию - они не несут никакой нагрузки для выявления клиентов, которые потенциально могут уйти, ко всему прочему при помощи ohe нужно будет преобразовать признаки: местоположение, пол

Далее стандартизируем данные, у нас будет много признаком значение, которым будет 0 или 1, нужно будет все оставшиеся признаки привести к виду среднее значение которых будет 0, а дисперсия 1, чтобы все было одного масштаба

In [10]:
features = data.drop(['row_number', 'customer_id', 'surname', 'exited'], axis=1)
target = data['exited']

### 1.4 Подготовим признаки

Воспользуемся методом get_dummies и разделим наши категории принципом OHE

Сделал столбец 'tenure' и 'num_of_products' типа object, чтобы его можно было разделить методом get_dummies, это категориальные признаки, хотя и выражены числами

In [11]:
features['tenure'] = features['tenure'].astype(object)

In [12]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=rnd_state
)

features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.15, random_state=rnd_state
)

features.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10


In [13]:
cat_features = ['geography', 'gender', 'tenure']

In [14]:
ohe_encoder = OneHotEncoder(drop='first', sparse=False)

train_temp = ohe_encoder.fit_transform(features_train[cat_features]) # Обучил ОНЕ инкодер
features_train[ohe_encoder.get_feature_names_out()] = train_temp # дал имена столбца, передал значения новым столбцам
features_train.drop(cat_features, axis=1, inplace=True) # Удалил категории, на основании которых делал ОНЕ

valid_temp = ohe_encoder.transform(features_valid[cat_features])
features_valid[ohe_encoder.get_feature_names_out()] = valid_temp 
features_valid.drop(cat_features, axis=1, inplace=True)

test_temp = ohe_encoder.transform(features_test[cat_features])
features_test[ohe_encoder.get_feature_names_out()] = test_temp 
features_test.drop(cat_features, axis=1, inplace=True)

C:\Users\gorok\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Распределение наших признаков с трудом можно назвать "нормальным", но стандартизацией я все таки воспользуюсь

In [15]:
cat_to_scale = ['credit_score', 'balance', 'estimated_salary', 'num_of_products', 'age']

In [16]:
scaler = StandardScaler()

features_train[cat_to_scale] = scaler.fit_transform(features_train[cat_to_scale])
features_valid[cat_to_scale] = scaler.transform(features_valid[cat_to_scale])
features_test[cat_to_scale] = scaler.transform(features_test[cat_to_scale])

### 1.5 Итоги подготовки данных

Изучили данные, привели в порядок названия столбцов, выбрали признаки, подготовили признаки, стандартизировали их, готовы двигаться дальше

## 2. Обучаем модель без учета дисбаланса, узучим баланс

### 2.1 Изучаем баланс

In [17]:
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Дисбаланс очень хорошо виден в расположении пользователей, во владении кредитной карты, количество продуктов у одного пользователя, хотя с этим признаком нужно быть поаккуратнее, в возрастной категории тоже виден яркий дисбаланс

### 2.2 Обучаем модель 

In [18]:
best_depth = 0
best_auc = 0
best_f1 = 0

for depth in range(1,20):
    model = DecisionTreeClassifier(class_weight='balanced', max_depth=depth, random_state=rnd_state) # Создаем модель
    model.fit(features_train, target_train) # Обучаем на тренировочной выборке
    result = model.predict(features_valid) # Предсказываем
    
    if f1_score(target_valid, result) > best_f1:
        best_depth = depth
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])
        
print(
      'depth =', best_depth, 
      'auc_roc =', best_auc, 
      'f1 =', best_f1
         )

depth = 4 auc_roc = 0.8378960830525659 f1 = 0.5953237410071942


Проверим модель на адекватность, сравним ее с моделью, которая будет предсказывать только положительный класс или только отрицательный

In [19]:
res = pd.Series(1, index=target_valid.index)
print('auc_roc =', roc_auc_score(target_valid, res), 
      'f1 =', f1_score(target_valid, res))

auc_roc = 0.5 f1 = 0.3518747424804285


Можно предположить, что решающее дерево неплохо себя показыват с глубиной - 5, все остальные результаты не сильно то и уходят от моделей, которые предсказывают только положительный класс

Посмотрим, что покажет случайный лес

In [20]:
%%time
best_depth = 0
best_auc = 0
best_f1 = 0
best_est = 0

for depth in range(1,80): # Ищем лучшую глубину
    model = RandomForestClassifier(max_depth=depth, random_state=rnd_state, n_estimators=20)
    model.fit(features_train, target_train)
    result = model.predict(features_valid)
    
    if f1_score(target_valid, result) > best_f1: # Промежуточный результат
        best_depth = depth
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])

CPU times: total: 36.5 s
Wall time: 38 s


In [21]:
%%time
for est in range(10,80): # Находим оптимальное количество ветвей 
    model = RandomForestClassifier(max_depth=best_depth, random_state=rnd_state, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.predict(features_valid)
    
    if f1_score(target_valid, result) > best_f1: # Финальный результат
        best_est = est
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])

CPU times: total: 1min 17s
Wall time: 1min 18s


In [22]:
print(
    'est =', best_est,
    'depth =', best_depth, 
    'auc_roc =', best_auc, 
    'f1 =', best_f1
    )

est = 57 depth = 23 auc_roc = 0.8561669329180506 f1 = 0.5671641791044776


оптимальная глубина дерева - 20

Решающее дерево показало результат совсем немногим хуже случайного леса

посмотри на логистическую регрессию

In [23]:
model = LogisticRegression(random_state=rnd_state, solver='liblinear')
model.fit(features_train, target_train)
result = model.predict(features_valid)
print('auc_roc =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]), 
      'f1 =', f1_score(target_valid, result))

auc_roc = 0.7578665745580798 f1 = 0.3020833333333333


Результаты не сильно радуют, возможно, решив проблему с балансом классов, мы сможем поменять картину, сейчас же наши метркии не сильно ушли от значений, которые можно получить предсказанием константы 

## 3 Балансируем классы

налаживаем баланс классов при помощи увеличения выборки 

In [24]:
features_zeros = features_train[target_train==0]
features_ones = features_train[target_train==1]

target_zeros = target_train[target_train==0]
target_ones = target_train[target_train==1]

features_upsampled = pd.concat([features_zeros]+[features_ones]*4)
target_upsampled = pd.concat([target_zeros]+[target_ones]*4)

features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

* На новых сбалансированных данных обучаем модель логистической регрессии 

In [25]:
model = LogisticRegression(class_weight='balanced', random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
result = model.predict(features_valid)

print('auc_roc =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]), 
      'f1 =', f1_score(target_valid, result))

auc_roc = 0.7636461898756981 f1 = 0.497883149872989


Пробуем подвинуть порог вхождения

In [26]:
probabilities_valid = model.predict_proba(features_valid) # Тут мы сохраняем вероятности вхождения жлементов в какой-либо класс
probabilities_one_valid = probabilities_valid[:,1] # Конкретезируем, что нам интересен положительный

In [27]:
for rng in np.arange(0.5, 0.57, 0.01):  # Смотрим, чтобы элементы были меньше "порога"
    predicted_valid = probabilities_one_valid > rng
    
    print(
         rng,
        'auc_roc =', roc_auc_score(target_valid, predicted_valid), 
        'f1 =', f1_score(target_valid, predicted_valid)
    )

0.5 auc_roc = 0.6980448761372756 f1 = 0.497883149872989
0.51 auc_roc = 0.6983299859603883 f1 = 0.5
0.52 auc_roc = 0.7023453446702328 f1 = 0.5061946902654867
0.53 auc_roc = 0.701793735325777 f1 = 0.5067385444743936
0.54 auc_roc = 0.7005900210073087 f1 = 0.507380073800738
0.55 auc_roc = 0.7012093718502065 f1 = 0.5093632958801497
0.56 auc_roc = 0.704371634326925 f1 = 0.5153256704980843


* Проделаем все то же самое для случайного леса

In [28]:
%%time
best_depth = 0  # сохраним лучшие результаты
best_auc = 0
best_f1 = 0
best_est = 20

for depth in range(1,20): # Найдем лучшую глубину при 20 ветвях и возьмем ее за эталонную
    model = RandomForestClassifier(max_depth=depth, random_state=rnd_state, n_estimators=60)
    model.fit(features_upsampled, target_upsampled)
    result = model.predict(features_valid)
    
    if f1_score(target_valid, result) > best_f1: # Определяем лучшие показатели на этом этапе 
        best_depth = depth
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])
print(best_depth, best_f1)


10 0.6460905349794239
CPU times: total: 23.3 s
Wall time: 23.7 s


In [29]:
%%time
for est in range(1,40): # Находим оптимальное количество ветвей
    model = RandomForestClassifier( max_depth=best_depth, random_state=rnd_state, n_estimators=est)
    model.fit(features_upsampled, target_upsampled)
    result = model.predict(features_valid)
    
    if f1_score(target_valid, result) > best_f1: # Финальный результат 
        best_est = est
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])

CPU times: total: 16.3 s
Wall time: 16.4 s


In [30]:
print(
    'est =', best_est,
    'depth =', best_depth, 
    'auc_roc =', best_auc, 
    'f1 =', best_f1
    )

est = 20 depth = 10 auc_roc = 0.862035133272093 f1 = 0.6460905349794239


* А теперь посмотрим на решающее дерево

In [31]:
best_depth = 0
best_auc = 0
best_f1 = 0

for depth in range(1,20):
    model = DecisionTreeClassifier(max_depth=depth, random_state=rnd_state) # Создаем модель
    model.fit(features_upsampled, target_upsampled) # Обучаем на тренировочной выборке
    result = model.predict(features_valid) # Предсказываем
    
    if f1_score(target_valid, result) > best_f1:
        best_depth = depth
        best_f1 = f1_score(target_valid, result)
        best_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])
        
print(
      'depth =', best_depth, 
      'auc_roc =', best_auc, 
      'f1 =', best_f1
         )

depth = 4 auc_roc = 0.8378960830525659 f1 = 0.5953237410071942


### Выводы

На валидационной выборке лучше всего себя проявил случайный лес, с параметрами глубины дерева 10 и количества ветвей 20

## 4 Финальное тестирование 

In [32]:
model = RandomForestClassifier(random_state=rnd_state, max_depth=10, n_estimators=20)
model.fit(features_upsampled, target_upsampled)
result = model.predict(features_test)

print(
     'auc_roc =', roc_auc_score(target_test, model.predict_proba(features_test)[:,1]), 
     'f1 =', f1_score(target_test, result)
)

auc_roc = 0.8430959089578454 f1 = 0.5900990099009901


Модель для проверки на адекватность

In [33]:
res = pd.Series(1, index=target_valid.index)
print('auc_roc =', roc_auc_score(target_valid, res), 
      'f1 =', f1_score(target_valid, res))

auc_roc = 0.5 f1 = 0.3518747424804285


Наша модель предсказывает положительный класс сильно лучше чем константная модель, а показатель аук-рок показыет, что общее число ошибок на тестовой выборке не превышает 20%

Используя данную модель можно предсказывать уход клинта с 59% точностью, эту информацию можно использовать для того, чтобы выявить "группу риска" и предпринять меры для их удержания